# Laboratorio #4 - Análisis de sentimientos
* Oscar Juárez - 17315
* José Pablo Cifuentes - 17509
* Paul Belches - 17088

In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
df = pd.read_csv('./files/GrammarandProductReviews.csv')
print([var for var in df])

['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'keys', 'manufacturer', 'manufacturerNumber', 'name', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'upc']


In [3]:
clean_df = df[['brand', 'manufacturer', 'name', 'reviews.username', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.rating', 'reviews.text']].copy()
clean_df

,brand,manufacturer,name,reviews.username,reviews.didPurchase,reviews.doRecommend,reviews.rating,reviews.text
0,Universal Music,Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),Joshua,NaN,NaN,5,i love this album. it's very good. more to the...
1,Lundberg,Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,Dorothy W,True,NaN,5,Good flavor. This review was collected as part...
2,Lundberg,Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,Dorothy W,True,NaN,5,Good flavor.
3,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Rebecca,False,False,1,I read through the reviews on here before look...
4,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Walker557,False,False,1,My husband bought this gel for us. The gel cau...
5,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Samantha,False,False,1,My boyfriend and I bought this to spice things...
6,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Raeanne,False,False,1,Bought this earlier today and was excited to c...
7,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Kimmie,False,False,1,I bought this product for my husband and I to ...
8,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,cassie,False,False,1,My husband and I bought this for some extra fu...
9,K-Y,K-Y,K-Y Love Sensuality Pleasure Gel,Moore222,False,False,1,Got as a surprise for my husband there is noth...


In [4]:
clean_reviews = []

for line in clean_df['reviews.text']:
    line = str(line)
    # Quitar saltos de linea y convertirlo a minusculas
    line = line.rstrip('\n').lower()
    # Quitar URLS
    line = re.sub(r'^https?:\/\/.[\r\n]', '', line)
    # Quitar el resto de expresiones regulares, excepto . ? ! y '
    line = re.sub(r"[^\w.?!\d'\s]", '', line)
    # Quitar números
    line = re.sub(' \d+', ' ', line)
    # Quitar espacios extra
    line = line.strip(' \t\n\r')
    # Reemplazar ! y ? por puntos.
    line = line.replace('!','.').replace('?','.')
    # Quitar multiples puntos por solo uno
    line = re.sub(r'\.+', ".", line)
    # Finalmente, quitamos apostrofes
    line = line.replace("'", '')

    clean_reviews.append(line)

### Validamos que no hayan usuarios o nombres de productos vacíos

Empezamos obteniendo la lista que contenga los índices de los valores vacíos. Luego, reemplazamos cada valor por '---'

In [5]:
# Creamos la lista de índices con valores vacíos
usernames = [str(prod) for prod in clean_df['reviews.username'].tolist()]
empties = np.where(pd.isnull(clean_df[['reviews.username']]))[0].tolist()

for index in empties:
    usernames[index] = '---'

Ahora, hacemos lo mismo para los productos.

In [6]:
products = [str(prod) for prod in clean_df['name'].tolist()]
empties = np.where(pd.isnull(clean_df[['reviews.username']]))[0].tolist()

for index in empties:
    products[index] = '---'

### Crear CSV con datos limpios

Primero, se cambian las columnas por aquellas que esten limpias.

In [7]:
clean_df['reviews.text'] = clean_reviews
clean_df['reviews.username'] = usernames
clean_df['name'] = products

Luego, crear CSV limpio

In [8]:
df = pd.DataFrame(clean_df)
df.to_csv('./files/CleanData.csv', index=False)

## Ejercicio 6



In [9]:
clean_df = pd.read_csv('./files/CleanData.csv')

### Inciso 6.1
¿Cuáles son los 10 productos de mejor calidad dado su review?

In [10]:
from PerWordAnalisys import perWordAnalisys
import csv
products = {}

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wuser\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
with open('files/CleanData.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if (row[2] in products):
                  products[row[2]].append(row[-1])
            else:
                  products[row[2]] = []
                  products[row[2]].append(row[-1])

Column names are brand, manufacturer, name, reviews.username, reviews.didPurchase, reviews.doRecommend, reviews.rating, reviews.text


In [12]:
reviewResults = {}
for count,product in enumerate(products):
    print(count, product)
    cont = 0
    for review in products[product]:
        cont += perWordAnalisys(review)
    reviewResults[product] = cont / len(products[product])

egetable & Noodle Soup
217 Maybelline Clean Express! Makeup Removing Lotion
218 Physicians Formula Powder Palette Mineral Glow Pearls, Translucent Pearl
219 Pleasant Hearth 7.5 Steel Grate, 30 5 Bar - Black
220 Switchmas (dvd)
221 Vaseline Intensive Care Healthy Hands Stronger Nails
222 Solo Foods Poppy Seed Cake and Pastry Filling
223 Alex Cross (dvdvideo)
224 Clear Scalp & Hair Therapy Total Care Nourishing Shampoo
225 Progresso Traditional Chicken Rice With Vegetables Soup
226 Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks
227 Bisquick Original Pancake And Baking Mix - 40oz
228 Olivella Bar Soap - 3.52 Oz
229 Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
230 Voortman Sugar Free Fudge Chocolate Chip Cookies
231 Finish Gelpacs Dishwasher Detergent, Orange Scent, 84ct
232 Tim Holtz Retractable Craft Pick-Red 6x.5
233 Ragu Traditional Pasta Sauce
234 Stargate (ws) (ultimate Edition) (director's Cut) (dvdvideo)
235 Iman Luxury Moisturizing Lipstick, Black Brandy 006

In [7]:
points = []
for i in reviewResults:
    points.append(reviewResults[i])
points.sort()

In [8]:
sort_products = sorted(reviewResults.items(), key=lambda x: x[1], reverse=True)
sort_products = sort_products[:10]
for count, i in enumerate(sort_products):
    print(count, i)

0 ('Elvis Presley - Girl Happy (cd)', 4.0)
1 ('JNH Lifestyles Goldstar 3 Person FAR Infrared Sauna', 4.0)
2 ('Healthy Skoop Organic Protein Powder - Chocolate - 15.9oz', 4.0)
3 ('Bebe Au Lait174 Premium Muslin Nursing Scarf - Nightingale', 4.0)
4 ('Avery174 11-1/4 X 9-1/4 Index Maker Extra Wide Label Dividers With 5 Tab - Clear (5 Sets Per Pack)', 3.0)
5 ('Carson-Dellosa Publishing Photographic Learning Cards Boxed Set, Nouns/verbs/adjectives, Grades K-12', 3.0)
6 ('Suave Kids Berry Body Wash', 3.0)
7 ('NYX Doll Eye Mascara, Waterproof - Black', 3.0)
8 ('Third Man On The Mountain (dvd)', 3.0)
9 ('Essex Pendulum Wall Clock Black - Timekeeper174', 3.0)


### Inciso 6.2
¿Cuáles son los 10 productos de menor calidad dado su review?

In [9]:
sort_products = sorted(reviewResults.items(), key=lambda x: x[1], reverse=False)
sort_products = sort_products[:10]
for count, i in enumerate(sort_products):
    print(count, i)

0 ('One Grace Place Terrific Tie Dye Throw Pillow Car (13x9x3)', -2.0)
1 ("Physician's Formula Cheek Palette Cream-to-Powder Blush Rose Palette", -1.0)
2 ('Horizon174 Organic Unsalted Butter - 1lbs', -1.0)
3 ('Orajel Severe P.M. Toothache Pain Relief Cream', -0.9285714285714286)
4 ('Moom Express Pre Waxed Strips For Face & Bikini', -0.2)
5 ('Lundberg Organic Cinnamon Toast Rice Cakes', 0.0)
6 ('Heinz Tomato Ketchup, 38oz', 0.0)
7 ('Pearhead Id Bracelet Frame', 0.0)
8 ('Craft Punch Giga Scallop Circle 45 24687534 To 334', 0.0)
9 ('Sea Gull Lighting Ceiling Fan - White', 0.0)


### Inciso 6.3
Cuáles son los usuarios que dan la mayor cantidad de reviews a distintos productos.

In [ ]:
# Diccionario de usuario -> producto
prod_user = clean_df[['name', 'reviews.username']].copy().values

Se crea un diccionario que tendrá al nombre del usuario como llave y la cantidad de productos comprados como valor.

In [ ]:
# Diccionario de todos los usuarios distintos. El valor será un arreglo de productos, sin repetición.
user_dict = {}

for prod, user in prod_user:

    # Si el usuario no está en el diccionario, agregarlo a él y el producto como valor.
    if user not in user_dict: 
        user_dict[user] = []
        user_dict[user].append(prod)

    # Si el usuario sí está en el diccionario y el producto no está en su lista de productos, agregarlo.
    elif prod not in user_dict[user]:
        user_dict[user].append(prod)

# Ahora, cambiar los productos comprados por la cantidad de productos.
for user, products in user_dict.items():
    user_dict[user] = len(products)

Se ordena el arreglo en base a los productos

In [ ]:
ordered = sorted(user_dict.items(), key=lambda x: x[1], reverse=True)

Se imprimen la primeras 7 posiciones

In [ ]:
print(ordered[0:7])

#### Respuesta:

Sin tomar en cuenta a los clientes "Anónimos":

1. Mike: 29 productos
2. Chris: 27 productos
3. Sandy: 23 productos
4. John: 23 productos
5. lisa: 22 productos

### Inciso 6.4
Cuáles son los usuarios que más reviews negativos y positivos dan en promedio.

Primero se lee el dataset y se cuenta la frecuencia de cada usuario:

In [13]:
from csv import reader
from PerWordAnalisys import *

#6.4  Cuáles son los usuarios que más reviews negativos y positivos dan en promedio.

usuariosPositivos={}
usuariosNegativos={}
usuariosTotales={}

# open file in read mode
with open('./files/CleanData.csv', encoding="utf8") as read_obj:
	# pass the file object to reader() to get the reader object
	csv_reader = reader(read_obj)

	contador=0

	# Iterate over each row in the csv using reader object
	for row in csv_reader:

		if(contador==0):
			contador+=1
			continue
		
		sentimiento=perWordAnalisys(row[-1])
		if(sentimiento>0):
			#Positivo
			try:
				usuariosPositivos[row[3]]+=1
				usuariosTotales[row[3]]+=1
			except Exception as e:
				usuariosPositivos[row[3]]=1
				usuariosTotales[row[3]]=1
			
			
		elif(sentimiento<0):
			try:
				usuariosNegativos[row[3]]+=1
				usuariosTotales[row[3]]+=1
			except Exception as e:
				usuariosNegativos[row[3]]=1
				usuariosTotales[row[3]]=1
		
			
		contador+=1

Luego, se saca el promedio para cada usuario

In [14]:
for usuario in usuariosTotales:
	try:
		usuariosPositivos[usuario]/=usuariosTotales[usuario]		
	except Exception as e:
		pass

	try:
		usuariosNegativos[usuario]/=usuariosTotales[usuario]		
	except Exception as e:
		pass

Finalmente se ordena y se imprimen top 5 usuarios de cada categoría

In [15]:
dictSortedPositivos=sorted(usuariosPositivos.items(), key=lambda x: x[1],reverse=True)
print("Top 5 usuarios que más reviews positivos dan en promedio:")
control=0
for i in dictSortedPositivos:
	control+=1
	print(i)
	if(control==5):
		break
print()
dictSortedNegativos=sorted(usuariosNegativos.items(), key=lambda x: x[1],reverse=True)
print("Top 5 usuarios que más reviews negativos dan en promedio:")
control=0
for i in dictSortedNegativos:
	control+=1
	print(i)
	if(control==5):
		break

Top 5 usuarios que más reviews positivos dan en promedio:
('---', 17.25)
('Sara', 8.0)
('Dawn', 6.0)
('Teresa', 5.5)
('Tammy', 5.0)

Top 5 usuarios que más reviews negativos dan en promedio:
('Cuca', 2.0)
('Nessa', 2.0)
('Walker557', 1.0)
('Jo276', 1.0)
('Star33', 1.0)


### Inciso 6.5
¿Cuáles son los productores que tienen productos de mejor calidad?

In [11]:
from PerWordAnalisys import perWordAnalisys
import csv
products = {}

In [12]:
with open('files/CleanData.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if (row[2] in products):
                  products[row[2]].append(row[-2])
            else:
                  products[row[2]] = []
                  products[row[2]].append(row[-2])

Column names are brand, manufacturer, name, reviews.username, reviews.didPurchase, reviews.doRecommend, reviews.rating, reviews.text


In [13]:
reviewResults = {}
for count,product in enumerate(products):
    print(count, product)
    cont = 0
    for rate in products[product]:
        cont += float(rate)
    reviewResults[product] = cont / len(products[product])

0 Pink Friday: Roman Reloaded Re-Up (w/dvd)
1 Lundberg Organic Cinnamon Toast Rice Cakes
2 K-Y Love Sensuality Pleasure Gel
3 J.R. Watkins Hand Cream, Lemon Cream
4 Ambi Complexion Cleansing Bar
5 Johnson's Baby Bubble Bath and Wash, 15oz
6 Olay Regenerist Deep Hydration Regenerating Cream
7 Windex Original Glass Cleaner Refill 67.6oz (2 Liter)
8 Heinz Tomato Ketchup, 38oz
9 Kind Dark Chocolate Chunk Gluten Free Granola Bars - 5 Count
10 Pantene Color Preserve Volume Shampoo, 25.4oz
11 Aussie Aussome Volume Shampoo, 13.5 Oz
12 Cars Toon: Mater's Tall Tales
13 CeraVe SA Renewing Cream
14 Solo Foods Almond Paste
15 Finish Quantum Dishwasher Detergent, Lemon Sparkle Scent, 45 Count
16 Disney174 Jake And The Neverland Pirates 4 Piece Bedding Set - Toddler
17 Pearhead Id Bracelet Frame
18 Craft Punch Giga Scallop Circle 45 24687534 To 334
19 Neutrogena Wet Skin Sunscreen Spray Broad Spectrum SPF 50, 5oz
20 Meguiar's Deep Crystal Car Wash 64-Oz.
21 Banana Boat Sunless Summer Color Self Tanni

In [14]:
points = []
for i in reviewResults:
    points.append(reviewResults[i])
points.sort()
print(points)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3475177304964538, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.3333333333333335, 2.4285714285714284, 2.4954128440366974, 2.5, 2.5185185185185186, 2.5714285714285716, 2.6, 2.6666666666666665, 2.6666666666666665, 2.75, 2.8014440433212995, 2.8111702127659575, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0434782608695654, 3.142857142857143, 3.1466666666666665, 3.1578947368421053, 3.1785714285714284, 3.3333333333333335, 3.3333333333333335, 3.3333333333333335, 3.3333333333333335, 3.3333333333333335, 3.3333333333333335, 3.35, 3.3870967741935485, 3.4, 3.4884488448844886, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.553435114503817, 3.5555555555555554, 3.5977011494252875, 3.6, 3.6, 3.626865671641791, 3.6470588235294117, 3.6666666666666665, 3.6666666666666665, 3.6666666666666665, 3.6666666666666665, 3.7, 3.7142857142857144, 3.7203389830508473, 3.75, 3.75, 3.75, 3.7591463414634148, 3.7659574468

In [15]:
sort_products = sorted(reviewResults.items(), key=lambda x: x[1], reverse=True)
sort_products = sort_products[:10]
for count, i in enumerate(sort_products):
    print(count, i)

0 ('Pink Friday: Roman Reloaded Re-Up (w/dvd)', 5.0)
1 ('Lundberg Organic Cinnamon Toast Rice Cakes', 5.0)
2 ('J.R. Watkins Hand Cream, Lemon Cream', 5.0)
3 ('Heinz Tomato Ketchup, 38oz', 5.0)
4 ('Solo Foods Almond Paste', 5.0)
5 ('Disney174 Jake And The Neverland Pirates 4 Piece Bedding Set - Toddler', 5.0)
6 ('Craft Punch Giga Scallop Circle 45 24687534 To 334', 5.0)
7 ('Citrus Magic Instant Spot & Stain Remover', 5.0)
8 ('La Tortilla Factory Hand Made Style Tortillas Flour', 5.0)
9 ("Black Sister's Revenge (dvd)", 5.0)


### Inciso 6.6
¿Cuáles son los productores que tienen productos de peor calidad?

In [16]:
sort_products = sorted(reviewResults.items(), key=lambda x: x[1], reverse=False)
sort_products = sort_products[:10]
for count, i in enumerate(sort_products):
    print(count, i)

0 ("Udi's Pepperoni Pizza", 1.0)
1 ('Pocket Watch Wall Clock Distressed Black - Yosemite Home Decor174', 1.0)
2 ('Kenroy Home Table Lamp - Chrome', 1.0)
3 ('High-Dome Floor Door Stop', 1.0)
4 ('Gone Fishing Titanized Fisherman Pliers with Braid Cutter', 1.0)
5 ('One Grace Place Terrific Tie Dye Throw Pillow Car (13x9x3)', 1.0)
6 ("Newman's Own Beef & Broccoli Complete Skillet Meal", 1.0)
7 ('No-Ad Sport Continuous Spray Sunblock, SPF 30', 1.0)
8 ('99179 - Fit System 01-05 Civic Replacement Mirror Glass, Driver Side - check description for fitment details', 1.0)
9 ('Dremel VC490 Velocity Universal Panel Cutting Blade', 1.0)
